In [1]:
!pip install tensorflow==1.12
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

     |████████████████████████████████| 83.1MB 76kB/s 
     |████████████████████████████████| 3.1MB 41.1MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

'1.12.0'

In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

nb_classes = 3

In [5]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.0681522 ,  0.9986474 , -0.5985123 ],
       [ 0.8480761 ,  0.03844468,  0.6499983 ],
       [ 0.21643075, -1.0498065 , -0.22993952],
       [ 0.27624148,  0.40409362,  0.25185513]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.496419  ,  0.73937017, -1.3157026 ], dtype=float32)>


In [6]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[8.0734241e-01 1.6441344e-01 2.8244188e-02]
 [9.5990753e-01 3.7002210e-02 3.0902838e-03]
 [9.5509523e-01 4.4351712e-02 5.5310916e-04]
 [9.9606007e-01 3.8962734e-03 4.3543489e-05]
 [9.9798977e-01 3.7353868e-05 1.9728271e-03]
 [9.9242711e-01 2.4183874e-03 5.1545538e-03]
 [9.9847037e-01 2.6901052e-05 1.5027093e-03]
 [9.9922574e-01 3.8378203e-06 7.7042897e-04]], shape=(8, 3), dtype=float32)


In [7]:
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[8.4479386e-01 1.5520594e-01 2.3555225e-07]], shape=(1, 3), dtype=float32)


In [8]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(4.82701, shape=(), dtype=float32)


In [9]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=112, shape=(4, 3), dtype=float32, numpy=
array([[ 1.4455996 , -0.70130694, -0.7442927 ],
       [ 1.6852416 , -1.1975796 , -0.48766208],
       [ 2.683767  , -1.8199471 , -0.86382025],
       [ 2.807219  , -1.9437262 , -0.8634931 ]], dtype=float32)>, <tf.Tensor: id=110, shape=(3,), dtype=float32, numpy=array([ 0.7133148 , -0.34348124, -0.36983353], dtype=float32)>]


In [10]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 2.277228
Loss at epoch 100: 0.686199
Loss at epoch 200: 0.571412
Loss at epoch 300: 0.509118
Loss at epoch 400: 0.461174
Loss at epoch 500: 0.419472
Loss at epoch 600: 0.380806
Loss at epoch 700: 0.343341
Loss at epoch 800: 0.305898
Loss at epoch 900: 0.269160
Loss at epoch 1000: 0.245441
Loss at epoch 1100: 0.232753
Loss at epoch 1200: 0.221375
Loss at epoch 1300: 0.211038
Loss at epoch 1400: 0.201602
Loss at epoch 1500: 0.192951
Loss at epoch 1600: 0.184990
Loss at epoch 1700: 0.177639
Loss at epoch 1800: 0.170833
Loss at epoch 1900: 0.164512
Loss at epoch 2000: 0.158627


In [11]:
a = hypothesis(x_data)

print(a)
print(tf.argmax(a, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[2.7395313e-06 1.0911912e-03 9.9890602e-01]
 [2.2300126e-03 8.5409790e-02 9.1236019e-01]
 [1.7348273e-07 1.6699983e-01 8.3300000e-01]
 [6.0755638e-06 8.4628570e-01 1.5370828e-01]
 [2.5640687e-01 7.3110116e-01 1.2491966e-02]
 [1.3445102e-01 8.6554158e-01 7.3822421e-06]
 [7.5148767e-01 2.4849071e-01 2.1564201e-05]
 [9.2009830e-01 7.9901412e-02 3.4291045e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# **Animal Classification**

In [19]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [20]:
nb_classes = 7
Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [21]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

In [22]:
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=Y)
    cost = tf.reduce_mean(cost_i)
    
    return cost

In [23]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [24]:

def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 9.204873085021973, Acc: 0.039603959769010544
Steps: 100 Loss: 0.4540657699108124, Acc: 0.8514851331710815
Steps: 200 Loss: 0.3149338662624359, Acc: 0.9108911156654358
Steps: 300 Loss: 0.24910101294517517, Acc: 0.9306930899620056
Steps: 400 Loss: 0.20628343522548676, Acc: 0.9504950642585754
Steps: 500 Loss: 0.17539048194885254, Acc: 0.9603960514068604
Steps: 600 Loss: 0.15200403332710266, Acc: 0.9900990128517151
Steps: 700 Loss: 0.13376285135746002, Acc: 1.0
Steps: 800 Loss: 0.11921367794275284, Acc: 1.0
Steps: 900 Loss: 0.10739220678806305, Acc: 1.0
Steps: 1000 Loss: 0.09763063490390778, Acc: 1.0
